In [ ]:
import numpy as np
import csv
import ast
import random

# TASK 1


In [2]:
def generate_legitimate_error():

    legitimate_error_list = []
    with open('dataset/legitimate_channel_error.csv', newline='') as csvfile:
        
        for line in csvfile:
            legitimate_error_list.append(ast.literal_eval(line.strip()))

    legitimate_error_list_clean = [elem[0] if isinstance(elem, tuple) else elem for elem in legitimate_error_list]

    return legitimate_error_list_clean


def generate_eavesdropper_error():

    eavesdropper_error_list = []
    with open('dataset/eavesdropper_channel_error.csv', newline='') as csvfile:
        
        for line in csvfile:
            eavesdropper_error_list.append(ast.literal_eval(line.strip()))

    eavesdropper_error_list_clean = [elem[0] if isinstance(elem, tuple) else elem for elem in eavesdropper_error_list]

    return eavesdropper_error_list_clean

print(generate_eavesdropper_error())



[[0, 0, 0, 0, 0, 0, 0], [1, 0, 0, 0, 0, 0, 0], [0, 1, 0, 0, 0, 0, 0], [0, 0, 1, 0, 0, 0, 0], [0, 0, 0, 1, 0, 0, 0], [0, 0, 0, 0, 1, 0, 0], [0, 0, 0, 0, 0, 1, 0], [0, 0, 0, 0, 0, 0, 1], [1, 1, 0, 0, 0, 0, 0], [1, 0, 1, 0, 0, 0, 0], [1, 0, 0, 1, 0, 0, 0], [1, 0, 0, 0, 1, 0, 0], [1, 0, 0, 0, 0, 1, 0], [1, 0, 0, 0, 0, 0, 1], [0, 1, 1, 0, 0, 0, 0], [0, 1, 0, 1, 0, 0, 0], [0, 1, 0, 0, 1, 0, 0], [0, 1, 0, 0, 0, 1, 0], [0, 1, 0, 0, 0, 0, 1], [0, 0, 1, 1, 0, 0, 0], [0, 0, 1, 0, 1, 0, 0], [0, 0, 1, 0, 0, 1, 0], [0, 0, 1, 0, 0, 0, 1], [0, 0, 0, 1, 1, 0, 0], [0, 0, 0, 1, 0, 1, 0], [0, 0, 0, 1, 0, 0, 1], [0, 0, 0, 0, 1, 1, 0], [0, 0, 0, 0, 1, 0, 1], [0, 0, 0, 0, 0, 1, 1], [1, 1, 1, 0, 0, 0, 0], [1, 1, 0, 1, 0, 0, 0], [1, 1, 0, 0, 1, 0, 0], [1, 1, 0, 0, 0, 1, 0], [1, 1, 0, 0, 0, 0, 1], [1, 0, 1, 1, 0, 0, 0], [1, 0, 1, 0, 1, 0, 0], [1, 0, 1, 0, 0, 1, 0], [1, 0, 1, 0, 0, 0, 1], [1, 0, 0, 1, 1, 0, 0], [1, 0, 0, 1, 0, 1, 0], [1, 0, 0, 1, 0, 0, 1], [1, 0, 0, 0, 1, 1, 0], [1, 0, 0, 0, 1, 0, 1], [1, 0, 0, 

In [3]:


def random_gen(n):
    if n <= 0:
        raise ValueError("n needs to be > 0")
    return random.randint(0, n - 1)


In [4]:
def binary_sum(x,y):

    result = []
    reminder = 0

    for i in reversed(range(len(x))):
        sum = x[i]+y[i]+reminder
        bit = sum%2
        reminder = sum // 2
        result.insert(0,bit)
    
    if reminder:
        result.insert(0,reminder)
    
    return result


In [5]:
def xor(x,y): #probabilmente si potrebbe usare numpy
    result = []
    for i in range(len(x)):
        result.append(x[i] ^ y[i])
    return result

In [6]:
def wiretap_channel(input):

    legitimate_error_list = generate_legitimate_error()
    eavesdropper_error_list = generate_eavesdropper_error()
    legitimate_error = legitimate_error_list[random_gen(8)]
    eavesdropper_error = eavesdropper_error_list[random_gen(64)]

    return (xor(input,legitimate_error),xor(input,eavesdropper_error))


x = [0,0,0,0,0,0,0]


In [7]:
def run_wiretap_simulation(input, num_runs=10**4):
    results = {}

    for _ in range(num_runs):
        y, z = wiretap_channel(input)

        # Convertiamo le liste x e y in tuple per una gestione più facile nel dizionario
        y_tuple = tuple(y)
        z_tuple = tuple(z)

        if (y_tuple, z_tuple) not in results:
            results[(y_tuple, z_tuple)] = 0
        results[(y_tuple, z_tuple)] += 1

    # Calcoliamo le probabilità
    probabilities = {key: count / num_runs for key, count in results.items()}
    
    return probabilities

# Funzione per dividere le probabilità per x e y
def divide_probabilities(probabilities):
    y_probabilities = {}
    z_probabilities = {}

    # Dividiamo le probabilità
    for (y, z), prob in probabilities.items():
        if y not in y_probabilities:
            y_probabilities[y] = 0
        if z not in z_probabilities:
            z_probabilities[z] = 0
        
        y_probabilities[y] += prob
        z_probabilities[z] += prob

    # Ordinamento delle probabilità
    y_probabilities = dict(sorted(y_probabilities.items()))
    z_probabilities = dict(sorted(z_probabilities.items()))

    return y_probabilities, z_probabilities




In [8]:
probabilities = run_wiretap_simulation(x)
x_probabilities, y_probabilities = divide_probabilities(probabilities)

In [9]:
# Stampa le probabilità ordinate per x e y
print("Probabilità per y:")
for x_val, prob in x_probabilities.items():
    print(f"x = {x_val} : {prob:.6f}")

print("\nProbabilità per z:")
for y_val, prob in y_probabilities.items():
    print(f"y = {y_val} : {prob:.6f}")

Probabilità per y:
x = (0, 0, 0, 0, 0, 0, 0) : 0.124600
x = (0, 0, 0, 0, 0, 0, 1) : 0.127600
x = (0, 0, 0, 0, 0, 1, 0) : 0.129900
x = (0, 0, 0, 0, 1, 0, 0) : 0.122900
x = (0, 0, 0, 1, 0, 0, 0) : 0.128800
x = (0, 0, 1, 0, 0, 0, 0) : 0.122700
x = (0, 1, 0, 0, 0, 0, 0) : 0.123400
x = (1, 0, 0, 0, 0, 0, 0) : 0.120100

Probabilità per z:
y = (0, 0, 0, 0, 0, 0, 0) : 0.014300
y = (0, 0, 0, 0, 0, 0, 1) : 0.017700
y = (0, 0, 0, 0, 0, 1, 0) : 0.018400
y = (0, 0, 0, 0, 0, 1, 1) : 0.015700
y = (0, 0, 0, 0, 1, 0, 0) : 0.016300
y = (0, 0, 0, 0, 1, 0, 1) : 0.016300
y = (0, 0, 0, 0, 1, 1, 0) : 0.015100
y = (0, 0, 0, 0, 1, 1, 1) : 0.014800
y = (0, 0, 0, 1, 0, 0, 0) : 0.015100
y = (0, 0, 0, 1, 0, 0, 1) : 0.015600
y = (0, 0, 0, 1, 0, 1, 0) : 0.016000
y = (0, 0, 0, 1, 0, 1, 1) : 0.016600
y = (0, 0, 0, 1, 1, 0, 0) : 0.016000
y = (0, 0, 0, 1, 1, 0, 1) : 0.014800
y = (0, 0, 0, 1, 1, 1, 0) : 0.015200
y = (0, 0, 1, 0, 0, 0, 0) : 0.015800
y = (0, 0, 1, 0, 0, 0, 1) : 0.014700
y = (0, 0, 1, 0, 0, 1, 0) : 0.013200

# Task 2

In [10]:
H_C = [[0,0,0,0,0,0,0],[0,1,0,0,1,0,1], [0,0,1,0,0,1,1], [0,0,0,1,1,1,1], [0,1,1,0,1,1,0], [0,1,0,1,0,1,0], [0,0,1,1,1,0,0], [0,1,1,1,0,0,1]]


def binary_complement(codeword):

    complement = []
    for i in range (len(codeword)):
        if(codeword[i]==1):
            complement.append(0)
        else:
            complement.append(1)
    return complement

def random_binning_encoder(u):
    mask = [1 for _ in range(7)]
    x = None

    for x in H_C:
        if x[1:4] == u:
            if random.randint(0, 1) == 1:
                x = xor(x, mask)    #calcolca il complemento
            break   
    return x

In [11]:
import itertools


all_messages = [list(vector) for vector in itertools.product([0, 1], repeat=3)]

for m in all_messages:
    print(f"Messaggio: {m}")
    x = random_binning_encoder(m)
    print(f"Codifica: {x}")

Messaggio: [0, 0, 0]
Codifica: [1, 1, 1, 1, 1, 1, 1]
Messaggio: [0, 0, 1]
Codifica: [0, 0, 0, 1, 1, 1, 1]
Messaggio: [0, 1, 0]
Codifica: [1, 1, 0, 1, 1, 0, 0]
Messaggio: [0, 1, 1]
Codifica: [0, 0, 1, 1, 1, 0, 0]
Messaggio: [1, 0, 0]
Codifica: [0, 1, 0, 0, 1, 0, 1]
Messaggio: [1, 0, 1]
Codifica: [0, 1, 0, 1, 0, 1, 0]
Messaggio: [1, 1, 0]
Codifica: [1, 0, 0, 1, 0, 0, 1]
Messaggio: [1, 1, 1]
Codifica: [0, 1, 1, 1, 0, 0, 1]


# Task 3

In [ ]:
def hamming_distance(x,y):
    
    if len(x) != len(y):
        raise ValueError("Le liste devono avere la stessa lunghezza")
    return sum(el_x != el_y for el_x, el_y in zip(x, y))


def random_binning_decoder(y):
    h_c = H_C + [xor([1,1,1,1,1,1,1],x) for x in H_C]
    distances = [hamming_distance(y, x) for x in h_c]
    min_index = distances.index(min(distances))

    x = h_c[min_index]
    if x[0] == 1:
        u = xor(x[1:4], [1,1,1])
    else:
        u = x[1:4]
    return u

In [13]:
all_messages = [list(vector) for vector in itertools.product([0, 1], repeat=3)]

for m in all_messages:
    print(f"Messaggio: {m}")
    x = random_binning_encoder(m)
    print(f"Codifica: {x}")

    u = random_binning_decoder(x)
    print(f"Decodifica: {u}")
    print("-----")

Messaggio: [0, 0, 0]
Codifica: [0, 0, 0, 0, 0, 0, 0]
Decodifica: [0, 0, 0]
-----
Messaggio: [0, 0, 1]
Codifica: [0, 0, 0, 1, 1, 1, 1]
Decodifica: [0, 0, 1]
-----
Messaggio: [0, 1, 0]
Codifica: [0, 0, 1, 0, 0, 1, 1]
Decodifica: [0, 1, 0]
-----
Messaggio: [0, 1, 1]
Codifica: [0, 0, 1, 1, 1, 0, 0]
Decodifica: [0, 1, 1]
-----
Messaggio: [1, 0, 0]
Codifica: [1, 0, 1, 1, 0, 1, 0]
Decodifica: [1, 0, 0]
-----
Messaggio: [1, 0, 1]
Codifica: [1, 0, 1, 0, 1, 0, 1]
Decodifica: [1, 0, 1]
-----
Messaggio: [1, 1, 0]
Codifica: [1, 0, 0, 1, 0, 0, 1]
Decodifica: [1, 1, 0]
-----
Messaggio: [1, 1, 1]
Codifica: [1, 0, 0, 0, 1, 1, 0]
Decodifica: [1, 1, 1]
-----


In [14]:
all_messages = [list(vector) for vector in itertools.product([0, 1], repeat=3)]

for m in all_messages:
    print(f"Messaggio: {m}")
    x = random_binning_encoder(m)
    print(f"Codifica: {x}")
    y, _ = wiretap_channel(x)
    print(f"Canale legittimo: {y}")

    u = random_binning_decoder(y)
    print(f"Decodifica: {u}")
    print("-----")

Messaggio: [0, 0, 0]
Codifica: [0, 0, 0, 0, 0, 0, 0]
Canale legittimo: [0, 0, 0, 0, 0, 0, 1]
Decodifica: [0, 0, 0]
-----
Messaggio: [0, 0, 1]
Codifica: [0, 0, 0, 1, 1, 1, 1]
Canale legittimo: [0, 0, 1, 1, 1, 1, 1]
Decodifica: [0, 0, 1]
-----
Messaggio: [0, 1, 0]
Codifica: [1, 1, 0, 1, 1, 0, 0]
Canale legittimo: [1, 1, 0, 0, 1, 0, 0]
Decodifica: [0, 1, 0]
-----
Messaggio: [0, 1, 1]
Codifica: [1, 1, 0, 0, 0, 1, 1]
Canale legittimo: [1, 0, 0, 0, 0, 1, 1]
Decodifica: [0, 1, 1]
-----
Messaggio: [1, 0, 0]
Codifica: [0, 1, 0, 0, 1, 0, 1]
Canale legittimo: [0, 1, 0, 0, 1, 0, 0]
Decodifica: [1, 0, 0]
-----
Messaggio: [1, 0, 1]
Codifica: [0, 1, 0, 1, 0, 1, 0]
Canale legittimo: [0, 0, 0, 1, 0, 1, 0]
Decodifica: [1, 0, 1]
-----
Messaggio: [1, 1, 0]
Codifica: [1, 0, 0, 1, 0, 0, 1]
Canale legittimo: [1, 0, 1, 1, 0, 0, 1]
Decodifica: [1, 1, 0]
-----
Messaggio: [1, 1, 1]
Codifica: [0, 1, 1, 1, 0, 0, 1]
Canale legittimo: [0, 1, 0, 1, 0, 0, 1]
Decodifica: [1, 1, 1]
-----


#TASK 5

In [15]:
def binary_symmetric_channel(input,error):
    output=[]
    for bit in input:
        if random.random()<=error:
                output.append(bit ^1)
        else:
                output.append(bit)

    return output

def evaluate_BSC(error=0.1,iterations=10**5):
    x = [0,0,0,0,0,0,0]
    #error = 0.1
    count = 0

    for i in range(iterations):
        x_bsc = binary_symmetric_channel(x,error)
        for i in range(7):
            if x_bsc[i]==1:
                count+=1

    print(f"Mean error for {iterations} iterations is: {count/(iterations*len(x))}.\nWith {count} errors!")

#Testing task 5

In [16]:
epsilon = 0.1
delta = 0.3
print("Legitimate channel:")
evaluate_BSC(epsilon,iterations=10**6)
print("Eavesdropper: ")
evaluate_BSC(delta,iterations=10**6)

Legitimate channel:
Mean error for 1000000 iterations is: 0.09952557142857144.
With 696679 errors!
Eavesdropper: 
Mean error for 1000000 iterations is: 0.29990442857142857.
With 2099331 errors!


In [36]:
def connected_channels(error,u=[0,0,0]):
    x=random_binning_encoder(u)
    y=binary_symmetric_channel(x,error)
    return random_binning_decoder(y)

def connected_channels_simulations(error,u=[0,0,0],iterations=10**5):
    
    count=0
    for i in range(iterations):
        if [0,0,0] != connected_channels(error):
            count+=1
    return (f"The number of incorrect decoding is: {count} over {iterations} iterations. With a probability of error in the decoding of {count/iterations}")


In [ ]:
print("Legitimate channel:")
connected_channels_simulations(epsilon)
print("Eavesdropper:")
connected_channels_simulations(delta)

'The number of incorrect decoding is: 66822 over 100000 iterations. With a probability of error in the decoding of 0.66822'

##TASK 6 

##TASK 7

In [84]:
def gray_encode(n):
    return n ^ (n >> 1)

def gray_decode(n):
    """Decodifica un numero in Gray code a binario"""
    mask = n.copy()
    while np.any(mask >> 1):
        mask = mask >> 1
        n = n ^ mask
    return n


def AWGN_channel(message=[-1,-3,0,-1,1,0,0]):

    noise = [np.random.normal() for i in range(7)]
    for i in range (len(message)):
        message[i]+=noise[i]
    return message+noise

def bits_to_integers(bits, bits_per_symbol):
    # Raggruppa i bit a gruppi di bits_per_symbol e converte in interi
    num_symbols = len(bits) // bits_per_symbol
    bits = bits[:num_symbols * bits_per_symbol]  # Troncamento se necessario
    bit_groups = bits.reshape((-1, bits_per_symbol))
    powers = 2 ** np.arange(bits_per_symbol - 1, -1, -1)
    return np.dot(bit_groups, powers)


def PAM_modulator(input, cardinality = 2**7):

    bits = np.array(input)
    bit_symbol = int(np.log2(cardinality))

    integers = bits_to_integers(bits, bit_symbol)

    integers = gray_encode(integers)

    pam_symbols = 2 * integers - (cardinality - 1)

    return pam_symbols

def PAM_demodulator(input, cardinality = 2**7):
        # 1. Livelli possibili PAM (es: M=4 → [-3, -1, +1, +3])
    levels = 2 * np.arange(cardinality) - (cardinality - 1)

    # 2. Trova l'indice del livello più vicino per ogni simbolo ricevuto
    indices = np.argmin(np.abs(input[:, None] - levels[None, :]), axis=1)
    
    indices = gray_decode(indices)

    # 3. Converti indici in bit
    bits_per_symbol = int(np.log2(cardinality))
    bit_matrix = ((indices[:, None] & (1 << np.arange(bits_per_symbol-1, -1, -1))) > 0).astype(int)
    bits = bit_matrix.flatten()

    return bits


In [81]:
bits = np.random.randint(0, 2, 63)
pam_symbols = PAM_modulator(bits)
bit_recieved =PAM_demodulator(pam_symbols)


print("Input bit:", bits)
print("Simboli PAM-128:", pam_symbols)
print(f"Output of demodulator: {bit_recieved}")


Input bit: [0 0 0 0 0 1 1 1 0 1 0 1 1 0 0 0 0 0 0 0 0 1 1 0 1 1 1 0 1 0 0 1 1 0 0 0 0
 0 0 0 0 1 0 0 0 0 1 0 0 1 1 0 1 0 1 1 1 0 0 1 1 0 1]
Simboli PAM-128: [-123  123 -127   51   85 -125 -115   61   87]
Output of demodulator: [0 0 0 0 0 1 1 1 0 1 0 1 1 0 0 0 0 0 0 0 0 1 1 0 1 1 1 0 1 0 0 1 1 0 0 0 0
 0 0 0 0 1 0 0 0 0 1 0 0 1 1 0 1 0 1 1 1 0 0 1 1 0 1]


In [91]:
def wiretap_AWGN_channel(u=[0,0,0]):
    x=random_binning_encoder(u)
    g = PAM_modulator(x)
    y = AWGN_channel(g)
    print(y)
    z = PAM_demodulator(y)
    return random_binning_decoder(z)

In [92]:
print(wiretap_AWGN_channel())

[-123.91787343 -125.162043   -124.92246606 -127.53022038 -127.11841442
 -123.57972826 -125.53930799]
[0 0 0 0 0 1 1 0 0 0 0 0 0 1 0 0 0 0 0 0 1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 1 1 0 0 0 0 0 0 1] [0, 0, 0, 0, 0, 0, 0]


ValueError: Le liste devono avere la stessa lunghezza